In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [3]:
df =pd.read_csv("2-CarPrice_Assignment.csv")

In [4]:
df.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [6]:
df.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [75]:
df.select_dtypes(include="object").columns

Index(['CarName', 'fueltype', 'aspiration', 'carbody', 'drivewheel',
       'enginelocation', 'enginetype', 'cylindernumber', 'fuelsystem'],
      dtype='object')

In [9]:
df["fueltype"].unique()

array(['gas', 'diesel'], dtype=object)

In [101]:
# One-Hot Encoding
#
#df_encoded =pd.concat([df, pd.get_dummies(df['fueltype'],prefix='fuel')],axis=1)
#df_encoded.drop('fueltype', axis=1, inplace=True)
df_encoded=df.copy()
categorical_cols = ["fueltype",'aspiration', 'drivewheel', 'fuelsystem']
for col in categorical_cols:
    df_encoded = pd.get_dummies(df_encoded, columns=[col], drop_first=True)
  
    one_hot_cols = [col for col in df_encoded.columns if df_encoded[col].dtype == 'bool']
    for c in one_hot_cols:
        df_encoded[c] = df_encoded[c].astype(int)

df_encoded['doornumber'] = df_encoded['doornumber'].map({'two': 2, 'four': 4})
df_encoded['enginelocation'] = df_encoded['enginelocation'].map({'front':0, 'rear':1})
df_encoded['cylindernumber'] = df_encoded['cylindernumber'].map({'four':4, 'six':6, 'five':5, 'three':3, 'twelve':12, 'two':2, 'eight':8})


In [91]:
df_encoded[["fuel_diesel","fuel_gas"]].astype(int)

KeyError: "None of [Index(['fuel_diesel', 'fuel_gas'], dtype='object')] are in the [columns]"

In [69]:
df_encoded['fuelsystem'].unique()

array(['mpfi', '2bbl', 'mfi', '1bbl', 'spfi', '4bbl', 'idi', 'spdi'],
      dtype=object)

In [60]:
# K-Fold target encoding
kf = KFold(n_splits=5, shuffle=True,random_state=42)
df_encoded['enginetype_encoded'] = np.nan


for train_index, val_index in kf.split(df):
    X_train, X_val = df_encoded.iloc[train_index], df_encoded.iloc[val_index]
    
    # train fold üzerinde enginetype ortalama price
    target_mean = X_train.groupby('enginetype')['price'].mean()
    
    # validation fold’u encode et
    df_encoded.loc[val_index, 'enginetype_encoded'] = X_val['enginetype'].map(target_mean)

median_encoded = df['enginetype_encoded'].median()
df_encoded['enginetype_encoded']=df_encoded['enginetype_encoded'].fillna(median_encoded)    

In [62]:
df_encoded["enginetype_encoded"].head()

0    18536.545455
1    16707.375000
2    25814.916667
3    11622.540233
4    11487.450568
Name: enginetype_encoded, dtype: float64

In [77]:
df_encoded.drop('enginetype', axis=1, inplace=True)

In [105]:
df_encoded.shape

(205, 34)

In [79]:
df_encoded.select_dtypes(include="object").columns

Index(['CarName', 'fueltype', 'aspiration', 'carbody', 'drivewheel'], dtype='object')

In [106]:
df_encoded["aspiration"].info()

KeyError: 'aspiration'